In [1]:
import pandas as pd
import numpy as np

import numba
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
hist_data = pd.read_csv("data/historical_user_logs.csv", parse_dates=["DateTime"])
hist_data.head()


,DateTime,user_id,product,action
0,2017-05-28 15:44:00,704,B,view
1,2017-05-29 07:08:00,499679,F,view
2,2017-05-29 07:10:00,499679,G,view
3,2017-05-29 07:10:00,499679,G,view
4,2017-05-29 07:10:00,499679,G,view


In [3]:
hist_data["hour"] = hist_data["DateTime"].dt.hour
hist_data["dayofweek"] = hist_data["DateTime"].dt.dayofweek
hist_data["dayofyear"] = hist_data["DateTime"].dt.dayofyear

In [4]:
action_map = {"view": 0, "interest": 1}
hist_data.action = hist_data.action.map(action_map)

In [5]:
prod_map = {"H": 0, "B": 1, "D": 2, "A": 3, "C": 4, "G": 5,
            "F": 6, "I": 7, "E": 8, "J": 9}
hist_data["product"] = hist_data["product"].map(prod_map)

In [6]:
hist_data.user_id.nunique()

171309

In [7]:
#user - count; count viewed; count inetrest; unique product viewed/ shown inetrest
#user, product - total count; count viewed/ shown inetrest

In [9]:
usr_data = hist_data.groupby("user_id").agg({"action": ["sum", "count"], "product": "nunique", 
                                            "hour": ["mean", "std"], "dayofweek":["mean", "std"],
                                            "dayofyear": ["min", "max", "mean", "std"]})
usr_data.columns = ["total_interest", "total_adclicks", "unique_prods", "hour_mean", "hour_std",
                   "dayofweek_mean", "dayofweek_std", "dayofyear_min", "dayofyear_max", "dayofyear_mean", "dayofyear_std"]
usr_data = usr_data.reset_index(drop=False)
usr_data.head()

,user_id,total_interest,total_adclicks,unique_prods,hour_mean,hour_std,dayofweek_mean,dayofweek_std,dayofyear_min,dayofyear_max,dayofyear_mean,dayofyear_std
0,4,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000
1,19,5,168,8,15.267857,5.396243,3.428571,1.897276,168,182,175.386905,4.431684
2,25,0,67,6,8.477612,2.414340,1.955224,2.395911,171,181,177.388060,1.906569
3,26,1,52,2,14.365385,8.794002,4.076923,1.643122,168,182,172.192308,4.589154
4,30,2,72,8,10.236111,7.930140,4.083333,1.607640,169,182,177.388889,5.025444


In [10]:
usr_prod_data = hist_data.groupby(["user_id", "product"]).agg({"action": ["sum", "count"]})
usr_prod_data.columns = ["usr_prod_interest", "usr_prod_adclicks"]
usr_prod_data = usr_prod_data.reset_index(drop=False)
usr_prod_data.head()

,user_id,product,usr_prod_interest,usr_prod_adclicks
0,4,0,0,1
1,4,1,0,1
2,4,3,0,3
3,4,7,0,2
4,19,0,0,16


In [11]:
all_data = pd.merge(usr_prod_data, usr_data, on=["user_id"], how="left")
all_data.head()

,user_id,product,usr_prod_interest,usr_prod_adclicks,total_interest,total_adclicks,unique_prods,hour_mean,hour_std,dayofweek_mean,dayofweek_std,dayofyear_min,dayofyear_max,dayofyear_mean,dayofyear_std
0,4,0,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000
1,4,1,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000
2,4,3,0,3,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000
3,4,7,0,2,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000
4,19,0,0,16,5,168,8,15.267857,5.396243,3.428571,1.897276,168,182,175.386905,4.431684


In [12]:
all_data["interest_ratio"] = all_data["total_interest"]/all_data["total_adclicks"]
all_data["product_interest_ratio"] = all_data["usr_prod_interest"] / all_data["usr_prod_adclicks"]

In [13]:
all_data.head()

,user_id,product,usr_prod_interest,usr_prod_adclicks,total_interest,total_adclicks,unique_prods,hour_mean,hour_std,dayofweek_mean,dayofweek_std,dayofyear_min,dayofyear_max,dayofyear_mean,dayofyear_std,interest_ratio,product_interest_ratio
0,4,0,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0
1,4,1,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0
2,4,3,0,3,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0
3,4,7,0,2,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0
4,19,0,0,16,5,168,8,15.267857,5.396243,3.428571,1.897276,168,182,175.386905,4.431684,0.029762,0.0


In [14]:
all_data["unq_prod_interest"] = all_data.user_id.map(hist_data.loc[hist_data["action"] == 1].groupby("user_id")["product"].nunique()).fillna(0)
all_data.head()

,user_id,product,usr_prod_interest,usr_prod_adclicks,total_interest,total_adclicks,unique_prods,hour_mean,hour_std,dayofweek_mean,dayofweek_std,dayofyear_min,dayofyear_max,dayofyear_mean,dayofyear_std,interest_ratio,product_interest_ratio,unq_prod_interest
0,4,0,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0
1,4,1,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0
2,4,3,0,3,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0
3,4,7,0,2,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0
4,19,0,0,16,5,168,8,15.267857,5.396243,3.428571,1.897276,168,182,175.386905,4.431684,0.029762,0.0,3.0


In [15]:
all_data["last_interest_days"]  = all_data.user_id.map(hist_data.loc[hist_data["action"] == 1].groupby("user_id")["dayofyear"].apply(lambda x: 182 - x.max())).fillna(0)
all_data.head()

,user_id,product,usr_prod_interest,usr_prod_adclicks,total_interest,total_adclicks,unique_prods,hour_mean,hour_std,dayofweek_mean,dayofweek_std,dayofyear_min,dayofyear_max,dayofyear_mean,dayofyear_std,interest_ratio,product_interest_ratio,unq_prod_interest,last_interest_days
0,4,0,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0
1,4,1,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0
2,4,3,0,3,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0
3,4,7,0,2,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0
4,19,0,0,16,5,168,8,15.267857,5.396243,3.428571,1.897276,168,182,175.386905,4.431684,0.029762,0.0,3.0,2.0


In [16]:
all_data["last_click_day"]  = all_data.user_id.map(hist_data.groupby("user_id")["dayofyear"].apply(lambda x: 182 - x.max())).fillna(0)

In [17]:
all_data.head()

,user_id,product,usr_prod_interest,usr_prod_adclicks,total_interest,total_adclicks,unique_prods,hour_mean,hour_std,dayofweek_mean,dayofweek_std,dayofyear_min,dayofyear_max,dayofyear_mean,dayofyear_std,interest_ratio,product_interest_ratio,unq_prod_interest,last_interest_days,last_click_day
0,4,0,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0,0
1,4,1,0,1,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0,0
2,4,3,0,3,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0,0
3,4,7,0,2,0,7,4,12.857143,0.899735,5.000000,0.000000,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0,0
4,19,0,0,16,5,168,8,15.267857,5.396243,3.428571,1.897276,168,182,175.386905,4.431684,0.029762,0.0,3.0,2.0,0


In [21]:
all_data["product_rank"] = all_data.groupby("user_id")["usr_prod_adclicks"].rank()
all_data.head()

,user_id,product,usr_prod_interest,usr_prod_adclicks,total_interest,total_adclicks,unique_prods,hour_mean,hour_std,dayofweek_mean,...,dayofyear_min,dayofyear_max,dayofyear_mean,dayofyear_std,interest_ratio,product_interest_ratio,unq_prod_interest,last_interest_days,last_click_day,product_rank
0,4,0,0,1,0,7,4,12.857143,0.899735,5.000000,...,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0,0,1.5
1,4,1,0,1,0,7,4,12.857143,0.899735,5.000000,...,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0,0,1.5
2,4,3,0,3,0,7,4,12.857143,0.899735,5.000000,...,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0,0,4.0
3,4,7,0,2,0,7,4,12.857143,0.899735,5.000000,...,182,182,182.000000,0.000000,0.000000,0.0,0.0,0.0,0,3.0
4,19,0,0,16,5,168,8,15.267857,5.396243,3.428571,...,168,182,175.386905,4.431684,0.029762,0.0,3.0,2.0,0,4.0


In [22]:
all_data.to_csv("data/hist_data_agg.csv", index=False)

In [23]:
agg2 = hist_data.groupby(["user_id", "dayofweek", "product"]).size()
agg2.name = "usr_dayofweek_clicks"
agg2 = agg2.reset_index(drop=False)
agg2.head()

,user_id,dayofweek,product,usr_dayofweek_clicks
0,4,5,0,1
1,4,5,1,1
2,4,5,3,3
3,4,5,7,2
4,19,0,1,2


In [24]:
agg3 = hist_data.groupby(["user_id", "hour", "product"]).size()
agg3.name = "usr_hourofday_clicks"
agg3 = agg3.reset_index()
agg3.head()

,user_id,hour,product,usr_hourofday_clicks
0,4,12,1,1
1,4,12,3,2
2,4,13,3,1
3,4,13,7,1
4,4,14,0,1


In [25]:
agg2.to_csv("data/agg2.csv", index=False)
agg3.to_csv("data/agg3.csv", index=False)

In [26]:
agg4 = hist_data.groupby(["user_id", "dayofweek"]).size()
agg4.name = "usr_dayofweek_cnt"
agg4 = agg4.reset_index(drop=False)
agg4.head()
agg4.to_csv("data/agg4.csv", index=False)

In [27]:
agg5 = hist_data.groupby(["user_id", "hour"]).size()
agg5.name = "usr_hour_cnt"
agg5 = agg5.reset_index(drop=False)
agg5.head()
agg5.to_csv("data/agg5.csv", index=False)

In [28]:
#from sklearn.decomposition import NMF, TruncatedSVD
tmp = hist_data.groupby(["user_id", "product"]).size().unstack().fillna(0)
tmp.head()

product,0,1,2,3,4,5,6,7,8,9
user_id,,,,,,,,,,
4,1.0,1.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0
19,16.0,21.0,53.0,14.0,1.0,19.0,36.0,0.0,8.0,0.0
25,10.0,0.0,0.0,8.0,0.0,42.0,2.0,4.0,1.0,0.0
26,24.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,32.0,8.0,2.0,2.0,14.0,1.0,0.0,7.0,6.0,0.0


In [29]:
nmf_feats = pd.DataFrame(np.log1p(tmp.values), columns=[f"prod_cnt{i+1}" for i in range(10)])
nmf_feats["user_id"] = tmp.index.values
print(nmf_feats.shape)
nmf_feats.head()

(171309, 11)


,prod_cnt1,prod_cnt2,prod_cnt3,prod_cnt4,prod_cnt5,prod_cnt6,prod_cnt7,prod_cnt8,prod_cnt9,prod_cnt10,user_id
0,0.693147,0.693147,0.000000,1.386294,0.000000,0.000000,0.000000,1.098612,0.000000,0.0,4
1,2.833213,3.091042,3.988984,2.708050,0.693147,2.995732,3.610918,0.000000,2.197225,0.0,19
2,2.397895,0.000000,0.000000,2.197225,0.000000,3.761200,1.098612,1.609438,0.693147,0.0,25
3,3.218876,3.367296,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,26
4,3.496508,2.197225,1.098612,1.098612,2.708050,0.693147,0.000000,2.079442,1.945910,0.0,30


In [30]:
nmf_feats.to_csv("data/nmf_feats.csv", index=False)

In [31]:
tmp = hist_data.groupby(["user_id", "dayofweek"]).size().unstack().fillna(0)
tmp.head()
nmf_feats = pd.DataFrame(np.log1p(tmp.values), columns=[f"day_cnt{i+1}" for i in range(7)])
nmf_feats["user_id"] = tmp.index.values
nmf_feats.shape
nmf_feats.to_csv("data/nmf_day_feats.csv", index=False)

In [32]:
user_time = hist_data.groupby(["user_id", "DateTime"]).size()
user_time.name = "user_time_counts"
user_time = user_time.reset_index()
user_time = user_time.groupby("user_id").agg({"user_time_counts": ["mean", "std", "max", "median", "skew"]})
user_time.columns = ["user_time_counts_mean", "user_time_counts_std", "user_time_counts_max",
                     "user_time_counts_median", "user_time_counts_skew"]
user_time = user_time.reset_index()
user_time.head()

,user_id,user_time_counts_mean,user_time_counts_std,user_time_counts_max,user_time_counts_median,user_time_counts_skew
0,4,1.000000,0.000000,1,1.0,0.000000
1,19,1.714286,1.084093,7,1.0,2.180657
2,25,1.634146,0.829340,4,1.0,1.069117
3,26,1.444444,0.808683,4,1.0,1.734458
4,30,1.469388,0.793254,4,1.0,1.540376


In [33]:
user_time.to_csv("data/user_time_counts.csv")

In [34]:
user_time.shape

(171309, 6)